### ===Task===

Let's modify the above scratch code to

- Modify the scratch code so it can accept an hyperparameter max_depth, in which it will continue create the tree until max_depth is reached.

- Put everything into a class DecisionTree. It should have at least two methods, fit(), and predict()

- Load the iris data and try with your class

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [3]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.n_features = X.shape[1]
        self.tree = self.grow_tree(X, y)
        
    def find_split(self, X, y):
        n_samples = y.size
        if n_samples <= 1:
            return None, None
        
        feature_ix, threshold = None, None
        sample_per_class_parent = [np.sum(y == c) for c in range(self.n_classes)]
        best_gini = 1.0 - sum((n / n_samples) ** 2 for n in sample_per_class_parent)
        for feature in range(self.n_features):
        
            # Sort data along selected feature.
            sample_sorted = sorted(X[:, feature]) #[2, 3, 10, 19]
            sort_idx = np.argsort(X[:, feature])
            y_sorted = y[sort_idx] #[0, 0, 1, 1]
                
            sample_per_class_left = [0] * self.n_classes   #[0, 0]
        
            sample_per_class_right = sample_per_class_parent.copy() #[2, 2]
            
            for i in range(1, n_samples):
                #the class of that sample
                c = y_sorted[i - 1]  #[0]
            
                #put the sample to the left
                sample_per_class_left[c] += 1  #[1, 0]
                        
                #take the sample out from the right  [1, 2]
                sample_per_class_right[c] -= 1
            
                gini_left = 1.0 - sum(
                    (sample_per_class_left[x] / i) ** 2 for x in range(self.n_classes)
                )
                        
                #we divided by n_samples - i since we know that the left amount of samples
                #since left side has already i samples
                gini_right = 1.0 - sum(
                    (sample_per_class_right[x] / (n_samples - i)) ** 2 for x in range(self.n_classes)
                )

                #weighted gini
                weighted_gini = ((i / n_samples) * gini_left) + ( (n_samples - i) /n_samples) * gini_right
            

                # in case the value are the same, we do not split
                # (both have to end up on the same side of a split).
                if sample_sorted[i] == sample_sorted[i - 1]:
                    continue

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    feature_ix = feature
                    threshold = (sample_sorted[i] + sample_sorted[i - 1]) / 2  # midpoint

        #return the feature number and threshold 
        #used to find best split
        return feature_ix, threshold

    def grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes)]
        #predicted class using the majority of sample class
        predicted_class = np.argmax(num_samples_per_class)
        #define the parent node
        node = Node(predicted_class=predicted_class)
        
        #From solution
        if depth < self.max_depth:
            
            #perform recursion
            feature, threshold = self.find_split(X, y)
            if feature is not None:
                #take all the indices that is less than threshold
                indices_left = X[:, feature] < threshold
                X_left, y_left = X[indices_left], y[indices_left]

                #tilde for negation
                X_right, y_right = X[~indices_left], y[~indices_left]

                #take note for later decision
                node.feature_index = feature
                node.threshold = threshold
                node.left = self.grow_tree(X_left, y_left, depth + 1)
                node.right = self.grow_tree(X_right, y_right, depth + 1)
        return node
        
    #to predict, it is as simple as moving 
    #through the tree 
    def _predict(self, sample):
        tree = self.tree
        while tree.left:
            if sample[tree.feature_index] < tree.threshold:
                tree = tree.left
            else:
                tree = tree.right
        return tree.predicted_class
    
    def predict(self, X):
        predict = [self._predict(sample) for sample in X]
        return predict

In [4]:
if __name__ == "__main__":
    import sys
    from sklearn.datasets import load_iris

    dataset = load_iris()
    X, y = dataset.data, dataset.target
    clf = DecisionTree(max_depth=10)
    clf.fit(X, y)
    print(clf.predict([[0, 0, 5, 1.5]]))

[2]


In [5]:
feature, threshold = clf.find_split(X, y)

print("Best feature used for split: ", feature)
print("Best threshold used for split: ", threshold)

Best feature used for split:  2
Best threshold used for split:  2.45
